<img src="https://github.com/FarzadNekouee/Keras-CIFAR10-CNN-Model/blob/master/image.png?raw=true" width="1800">

The __CIFAR-10__ dataset is a well-established benchmark in the field of machine learning, specifically designed for __image classification__. Comprising __60,000 color images__, each of __size 32x32 pixels__, the dataset is segmented into __10 distinct classes__, each representing a different object or creature. The classes encompass the following:

- Airplane
- Automobile
- Bird
- Cat
- Deer
- Dog
- Frog
- Horse
- Ship
- Truck

Each class contains an equal distribution, boasting 6,000 images. From the total image count, 50,000 are designated for training while the remaining 10,000 are set aside for testing.

First, we're importing all the necessary libraries to kick off our project. We'll be relying on __TensorFlow__ and __Keras__ to handle the image data, craft our model, and optimize it for best performance:

In [ ]:
import cv2
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import load_model

 # Data Preparation and Exploration

Let's download the __CIFAR-10__ dataset from Keras library, and then split original training data to training and validation sets:

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

  Now, we're printing out the dimensions of our training, validation, and test datasets:

In [ ]:
print('Train Images Shape:      ', X_train.shape)
print('Train Labels Shape:      ', y_train.shape)

print('\nValidation Images Shape: ', X_valid.shape)
print('Validation Labels Shape: ', y_valid.shape)

print('\nTest Images Shape:       ', X_test.shape)
print('Test Labels Shape:       ', y_test.shape)

Afterward, let's take an overview of the __CIFAR-10__ dataset:

In [ ]:
# CIFAR-10 classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Create a new figure
plt.figure(figsize=(15,15))

# Loop over the first 25 images
for i in range(64):
    # Create a subplot for each image
    plt.subplot(8, 8, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    # Display the image
    plt.imshow(X_train[i])

    # Set the label as the title
    plt.title(class_names[y_train[i][0]], fontsize=12)

# Display the figure
plt.show()

# 3: Data Preprocessing
In the **Data Preprocessing** phase, we undertake essential preparatory measures to ensure our dataset is aptly primed for the modeling process:

1. **Normalization of Image Data**

2. **One-Hot Encoding of Labels**

3. **Data Augmentation**

   
Convert the pixel values data type to __float32__ type, and then normalizes them by subtracting the mean and dividing by the standard deviation of the training set, enhancing the model's training efficiency and effectiveness:

In [ ]:
# Convert pixel values data type to float32
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')
X_valid = X_valid.astype('float32')

# Calculate the mean and standard deviation of the training images
mean = np.mean(X_train)
std  = np.std(X_train)

# Normalize the data
# The tiny value 1e-7 is added to prevent division by zero
X_train = (X_train-mean)/(std+1e-7)
X_test  = (X_test-mean) /(std+1e-7)
X_valid = (X_valid-mean)/(std+1e-7)

Convert the class labels to one-hot vectors to transform the categorical labels into a format suitable for multi-class classification by neural networks:

In [ ]:
y_train = to_categorical(y_train, 10)
y_valid = to_categorical(y_valid, 10)
y_test  = to_categorical(y_test, 10)

Finally, implement data augmentation to artificially expand the size of the training set by creating modified versions of images in the dataset. This helps improve the model's ability to generalize, thereby reducing overfitting. Data augmentation techniques such as rotations, shifts, flips, shearing, and intensity changes introduce small variations to the existing images, creating a broader set of training samples to learn from.

The choice of data augmentation techniques often depends on the specific characteristics of the dataset and the problem at hand. The __CIFAR-10__ dataset comprises small color images of objects from 10 different classes. Given the nature of these images, some augmentation techniques are more applicable than others:

* __Rotation__: A small degree of rotation can help the model become invariant to the orientation of the object. The `rotation_range=15` means the image could be rotated randomly within -15 to 15 degrees. However, large rotations could be harmful since the CIFAR-10 images are relatively small and a big rotation might put the object outside of the image.

    
* __Width and Height shift__: Small shifts can help the model become invariant to the position of the object in the image. Here, `width_shift_range=0.12` and `height_shift_range=0.12` mean the image could be moved horizontally or vertically by up to 12% of its width or height respectively. Again, since the images are small, large shifts might put the object outside of the image.

    
* __Horizontal Flip__: A horizontal flip is a sensible choice for this dataset because for many images, the object of interest remains the same when flipped horizontally (for example, a flipped car is still a car).

    
* __Zoom__: Small zooming in by up to 10% (`zoom_range=0.1`) can also help the model generalize better. However, excessive zooming might lead to significant information loss.

    
* __Brightness Range__: Changing brightness can simulate various lighting conditions. With `brightness_range=[0.9,1.1]`, the brightness of the image is randomly changed to a value between 90% and 110% of the original brightness.

    
* __Shear Intensity__: With `shear_range=10`, a shear intensity within the range of -10 to +10 degrees is applied. This transformation slants the shape of the image, helping the model to recognize objects in different perspectives.

    
* __Channel Shift Intensity__: With `channel_shift_range=0.1`, the intensities of the RGB channels are randomly shifted by up to 10% of their full scale. This can help the model handle different lighting conditions and color variations.

While some augmentation techniques like vertical flips and color jittering may not be appropriate for all classes in the __CIFAR-10__ dataset, the chosen techniques are expected to help improve the robustness and generalization capability of the model.

In [ ]:
# Data augmentation
data_generator = ImageDataGenerator(
    # Rotate images randomly by up to 15 degrees
    rotation_range=15,

    # Shift images horizontally by up to 12% of their width
    width_shift_range=0.12,

    # Shift images vertically by up to 12% of their height
    height_shift_range=0.12,

    # Randomly flip images horizontally
    horizontal_flip=True,

    # Zoom images in by up to 10%
    zoom_range=0.1,

    # Change brightness by up to 10%
    brightness_range=[0.9,1.1],

    # Shear intensity (shear angle in counter-clockwise direction in degrees)
    shear_range=10,

    # Channel shift intensity
    channel_shift_range=0.1,
)

   When running the above code, we are setting up a pipeline for preprocessing the images during model training. The generator applies a series of random transformations (specified by the parameters) to the images each time they are loaded into the model for training. Each time an epoch is run during model training, these random transformations will create different variations of the original images. These changes are made __on-the-fly__ and don't modify our original dataset.      

# 4: Define CNN Model Architecture

In [ ]:
# our model

Another model architecture that will be defined is inspired from the __VGG16__ network. It contains multiple convolutional layers followed by max-pooling and dropout layers, and finally a fully connected layer for classification. While not adopting advanced modules like residuals or inceptions, this design is simpler, ensuring fewer parameters and a more straightforward architecture, making it more computationally efficient.

Here is a brief explanation of the architecture:

- The network begins with __a pair of Conv2D layers__, each with __32 filters of size 3x3__. This is followed by a __Batch Normalization__ layer which accelerates training and provides some level of __regularization__, helping to prevent overfitting.

    
- The pairs of Conv2D layers are followed by a __MaxPooling2D layer__, which reduces the spatial dimensions (height and width), effectively providing a form of translation invariance and reducing computational complexity. This is followed by a __Dropout layer__ that randomly sets a fraction (0.2 for the first dropout layer) of the input units to 0 at each update during training, helping to prevent overfitting.

    
- This pattern of two Conv2D layers, followed by a Batch Normalization layer, a MaxPooling2D layer, and a Dropout layer, repeats three more times. The number of filters in the Conv2D layers doubles with each repetition, starting from 32 and going up to 64, 128, and then 256. This increasing pattern helps the network to learn more complex features at each level. The dropout rate also increases at each step, from 0.2 to 0.5.

    
- After the convolutional and pooling layers, a __Flatten layer__ is used to convert the 2D outputs of the preceding layer into a 1D vector.

    
- Finally, a __Dense (or fully connected) layer__ is used for classification. It has 10 units, each representing one of the 10 classes of the CIFAR-10 dataset, and a __softmax activation function__ is used to convert the outputs to probability scores for each class.

    
This architecture leverages the strengths of deep CNNs to learn hierarchical features from the CIFAR-10 images. Regularization techniques such as __L2 regularization__, __Dropout__, and __Batch Normalization__ are also used to combat overfitting. While being inspired by VGG16, the model remains simpler and does not incorporate the more advanced features of recent architectures, focusing instead on efficiency and simplicity.

In [ ]:
# Initialize a sequential model
model = Sequential()

# Set the weight decay value for L2 regularization
weight_decay = 0.0001

# Add the first convolutional layer with 32 filters of size 3x3
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay),
                 input_shape=X_train.shape[1:]))
# Add batch normalization layer
model.add(BatchNormalization())

# Add the second convolutional layer similar to the first
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())

# Add the first max pooling layer with pool size of 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))
# Add dropout layer with 0.2 dropout rate
model.add(Dropout(rate=0.2))

# Add the third and fourth convolutional layers with 64 filters
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())

# Add the second max pooling layer and increase dropout rate to 0.3
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.3))

# Add the fifth and sixth convolutional layers with 128 filters
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())

# Add the third max pooling layer and increase dropout rate to 0.4
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.4))

# Add the seventh and eighth convolutional layers with 256 filters
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_regularizer=l2(weight_decay)))
model.add(BatchNormalization())

# Add the fourth max pooling layer and increase dropout rate to 0.5
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.5))

# Flatten the tensor output from the previous layer
model.add(Flatten())

# Add a fully connected layer with softmax activation function for outputting class probabilities
model.add(Dense(10, activation='softmax'))

Let's display the detailed architecture of the model:

In [ ]:
model.summary()

Our model consists of __1,186,346 parameters__, of which 1,184,426 are trainable. This is __a relatively compact model__, especially when compared to advanced architectures which often have tens or even hundreds of millions of parameters.

# 5 Training the CNN Model

The training uses a batch size of 64 and will run for a maximum of 250 epochs or until the early stopping condition is met. During the training, the model's performance is evaluated on the validation data after each epoch. I've added a couple of callback functions to enhance the training process:

* The __ReduceLROnPlateau callback__ is used to reduce the learning rate by half (factor=0.5) whenever the validation loss does not improve for 10 consecutive epochs. This helps to adjust the learning rate dynamically, allowing the model to get closer to the global minimum of the loss function when progress has plateaued. This strategy can improve the convergence of the training process.
    

* The __EarlyStopping callback__ is employed to monitor the validation loss and halt the training process when there hasn't been any improvement for a certain number of epochs, ensuring that the model doesn't waste computational resources and time. Furthermore, this callback restores the best weights from the training process, ensuring we conclude with the optimal model configuration from the epochs.

In [ ]:
# Set the batch size for the training
batch_size = 64

# Set the maximum number of epochs for the training
epochs = 300

# Define the optimizer (Adam)
optimizer = Adam(learning_rate=0.0005)

# Compile the model with the defined optimizer, loss function, and metrics
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Add ReduceLROnPlateau callback
# Here, the learning rate will be reduced by half (factor=0.5) if no improvement in validation loss is observed for 10 epochs
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)

# Add EarlyStopping callback
# Here, training will be stopped if no improvement in validation loss is observed for 40 epochs.
# The `restore_best_weights` parameter ensures that the model weights are reset to the values from the epoch
# with the best value of the monitored quantity (in this case, 'val_loss').
early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True, verbose=1)

# Fit the model on the training data, using the defined batch size and number of epochs
# The validation data is used to evaluate the model's performance during training
# The callbacks implemented are learning rate reduction when a plateau is reached in validation loss and
# stopping training early if no improvement is observed
model.fit(data_generator.flow(X_train, y_train, batch_size=batch_size),
          epochs=epochs,
          validation_data=(X_valid, y_valid),
          callbacks=[reduce_lr, early_stopping],
          verbose=2)

# 6 Visualizing the Learning Curves

Generate plots for visualizing the training and validation loss, and accuracy evolution over epochs using model history:

In [ ]:
plt.figure(figsize=(15,6))

# Plotting the training and validation loss
plt.subplot(1, 2, 1)
plt.plot(model.history.history['loss'], label='Train Loss', color='#8502d1')
plt.plot(model.history.history['val_loss'], label='Validation Loss', color='darkorange')
plt.legend()
plt.title('Loss Evolution')

# Plotting the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(model.history.history['accuracy'], label='Train Accuracy', color='#8502d1')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy', color='darkorange')
plt.legend()
plt.title('Accuracy Evolution')

plt.show()

Based on the visualizations above, it's evident that the model is performing well without signs of overfitting. This conclusion is supported by the close alignment of training and validation accuracy and loss values throughout the training process. The gap between training and validation accuracy remains minimal, indicating that the model generalizes well to unseen data. Similarly, the model's loss on validation data closely follows the training loss, reinforcing the assertion of good generalization. Therefore, the model appears to be well regularized and not overfitting to the training data.

# 7 Evaluating the Optimal Model on Test Data

As we have set `restore_best_weights=True` in EarlyStopping, after training, the model itself will have the best weights Following this, I will use this model to evaluate its performance on the test data, calculating the test accuracy and loss:

In [ ]:
# Use the model to make predictions, evaluate on test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

print('\nTest Accuracy:', test_acc)
print('Test Loss:    ', test_loss)

With a test accuracy of __more than 90%__, our model demonstrates exceptional performance on unseen data. This high accuracy, achieved using __a relatively compact model of just about 1.2 million parameters__, is noteworthy. Many advanced architectures employ tens or even hundreds of millions of parameters to achieve similar or only slightly better results. The proximity of the test loss and accuracy to their respective training counterparts signifies that our model is not merely memorizing the training data but is genuinely understanding patterns and effectively generalizing from the training data to unseen data. Thus, it can be inferred that our model not only delivers reliable predictions but also strikes a balance between efficiency and performance.

# 8 Performance on an Out-of-Dataset Image

To further explore the generalization capability of our trained CIFAR-10 classification model, we can assess its performance using an external truck image. This image, which isn't part of the CIFAR-10 dataset, has been sourced from [this GitHub repository](https://github.com/FarzadNekouee/Keras-CIFAR10-CNN-Model/blob/master/truck_sample.png).

In [ ]:
# Fetch the raw image from GitHub
url = "https://raw.githubusercontent.com/FarzadNekouee/Keras-CIFAR10-CNN-Model/master/truck_sample.png"
resp = urllib.request.urlopen(url)
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)

# Convert the image from BGR to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

Let's see our desired image:

In [ ]:
# Display the image
plt.imshow(image)
plt.xticks([])
plt.yticks([])
plt.grid(False)
plt.show()

   We need to preprocess it in the same way as we did with the training and test data:

In [ ]:
# Resize it to 32x32 pixels
image = cv2.resize(image, (32,32))

# Normalize the image
image = (image-mean)/(std+1e-7)

# Add an extra dimension because the model expects a batch of images
image = image.reshape((1, 32, 32, 3))

Make the prediction:

In [ ]:
prediction = model.predict(image)

   The prediction will be a 10-element probability vector. To get the predicted class, we find the index with the maximum value:

In [ ]:
predicted_class = prediction.argmax()

print('Predicted class: ', class_names[predicted_class])